# Optimize Starlet thresholds using Differential Evolution

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/pywi-cta-notebooks/master?filepath=tuto_4d_optimize_starlet_thresholds_using_differential_evolution.ipynb)

In [ ]:
import pywicta
print(pywicta.get_version())

In [ ]:
import json

from pywicta.optimization.objectivefunc.starlet import ObjectiveFunction
from pywicta.data.lst_gamma_prod3b_north_faint import data_dir

from scipy import optimize

import pywicta.denoising.cdf
from pywicta.denoising.inverse_transform_sampling import EmpiricalDistribution

In [ ]:
search_ranges = ((1., 5.),     # Scale 0 (smallest scale)
                 (1., 5.))     # Scale 1 (largest scale)

In [ ]:
#instrument = "ASTRICam"
#instrument = "CHEC"
#instrument = "DigiCam"
#instrument = "FlashCam"
#instrument = "NectarCam"
instrument = "LSTCam"

cleaning_failure_score = 90.
#cleaning_failure_score = float('nan')

input_files = [data_dir]    # Write here the path of the directory where images are stored

#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.ASTRI_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.GCT_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.DIGICAM_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.FLASHCAM_CDF_FILE)
#noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.NECTARCAM_CDF_FILE)
noise_distribution = EmpiricalDistribution(pywicta.denoising.cdf.LSTCAM_CDF_FILE)

func = ObjectiveFunction(input_files=input_files,
                         cam_id=instrument,
                         max_num_img=10,                         # integer or None           should be at least 1000 in production
                         noise_distribution=noise_distribution,
                         aggregation_method="mean",              # "mean" or "median"
                         num_scales=len(search_ranges) + 1,      # TODO: useless now...
                         type_of_filtering="cluster_filtering",
                         last_scale_treatment="mask",
                         detect_only_positive_structures=False,
                         kill_isolated_pixels=False,
                         #tmp_files_directory="/dev/shm/.jd/",
                         cleaning_failure_score=cleaning_failure_score)

In [ ]:
x_list = []
fx_list = []

def callback(xk, convergence):
    x_list.append(xk.tolist())
    fx_list.append(float(func(xk)))

    fx_best = min(fx_list)
    fx_best_index = fx_list.index(fx_best)
    x_best = x_list[fx_best_index]

    print("{}: f({})={} ({}) ; best ({}): f({})={}".format(len(x_list), x_list[-1], fx_list[-1], convergence, fx_best_index, x_best, fx_best))

    res_dict = {
                "best_solution": x_best,
                "best_score": float(fx_best),
                "solutions": x_list,
                "scores": fx_list
               }

    with open("optimize_sigma_diff_evo.json", "w") as fd:
        json.dump(res_dict, fd, sort_keys=True, indent=4)  # pretty print format

res = optimize.differential_evolution(func,
                                      search_ranges,
                                      maxiter=10,        # The number of iterations
                                      popsize=5,
                                      callback=callback,
                                      #polish=False,
                                      disp=False)          # Print status messages

print()
print("best solution:", res.x)
print("best score:", res.fun)

#print("Cause of the termination:", res.message)
#print("Number of evaluations of the objective functions:", res.nfev)
#print("Number of iterations performed by the optimizer:", res.nit)